In [ ]:
home = '/mnt/f/YOLOv8-DeepSORT-Object-Tracking/ultralytics/yolo/v8/detect'
%cd {home}

In [ ]:
!python predict_count.py model=yolov8n.pt source=/mnt/f/YOLOv8-DeepSORT-Object-Tracking/sample_video.mp4 +project=. +name=output ++save=True

In [ ]:
import gradio as gr
import shutil
import os
import uuid
from pathlib import Path
import subprocess

# 路径定义
PREDICT_SCRIPT = "/mnt/f/YOLOv8-DeepSORT-Object-Tracking/ultralytics/yolo/v8/detect/predict_count.py"
OUTPUT_DIR = "/mnt/f/YOLOv8-DeepSORT-Object-Tracking/runs/detect/train"
BEST_WEIGHTS = "/mnt/f/YOLOv8-DeepSORT-Object-Tracking/ultralytics/yolo/v8/detect/yolov8l.pt"

def process_video(input_video):
    # 创建唯一文件名
    temp_input = f"inputs/{uuid.uuid4().hex}.mp4"
    os.makedirs("inputs", exist_ok=True)
    shutil.copy(input_video, temp_input)
    fixed_output_dir = Path("/mnt/f/YOLOv8-DeepSORT-Object-Tracking/runs/detect/train")
    if fixed_output_dir.exists():
        shutil.rmtree(fixed_output_dir)
    # 执行命令
    command = [
        "python", PREDICT_SCRIPT,
        f"model={BEST_WEIGHTS}",
        f"source={temp_input}",
        "save=True",
        "conf=0.5",
        f"+project={'/mnt/f/YOLOv8-DeepSORT-Object-Tracking/runs/detect'}"
    ]
    result = subprocess.run(command, capture_output=True, text=True)

    print("=== predict.py 输出日志 ===")
    print(result.stdout)
    print("=== 错误信息（如果有） ===")
    print(result.stderr)

    if result.returncode != 0:
        raise RuntimeError("predict.py 执行失败，请检查上方错误信息。")

    # 查找最新输出文件夹
    fixed_output_dir = Path("/mnt/f/YOLOv8-DeepSORT-Object-Tracking/runs/detect/train")

    # 查找 MP4 文件
    output_path = ""
    for f in fixed_output_dir.glob("*.mp4"):
        output_path = str(f)
        break

    if not output_path or not os.path.exists(output_path):
        raise FileNotFoundError(
            f"未能在 {fixed_output_dir} 找到输出视频文件，请检查 predict.py 是否正确保存结果。"
        )

    return output_path

# Gradio界面
demo = gr.Interface(
    fn=process_video,
    inputs=gr.Video(label="上传视频"),
    outputs=gr.Video(label="处理后的视频"),
    title="YOLOv8 + DeepSort 视频目标检测",
    description="上传视频，模型将检测和跟踪目标并输出处理后的视频结果。"
)

if __name__ == "__main__":
        demo.launch(
        allowed_paths=["/mnt/f/YOLOv8-DeepSORT-Object-Tracking/runs/detect/train"],
        max_file_size="2000mb"         
         )
